# The ***GroupBy*** Pattern

We have seen the ***GroupBy*** operator in ***Pandas***, but  this is actually a more general ***design pattern*** that can be utilized in many data analyics frameworks and data access interfaces, e.g. in ***SQL***. 

## GroupBy: general Pattern
<img SRC="IMG/groupby.jpg">

#### GroupBy in SQL:
```
SELECT COUNT(CustomerID), Country
FROM Customers
GROUP BY Country
ORDER BY COUNT(CustomerID) DESC;
```

#### GroupBy in MongoDB
```
db.BusinessProcess.aggregate({
    "$group": {
        _id: {
            status: "$status",
            type: "$type"
        },
        count: {
            $sum: 1
        }
    }
   })
```


In [1]:
#setup example
import numpy as np
import pandas as pd
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

key1 key2     data1     data2
0    a  one  0.267529  0.828211
1    a  two  1.501438  1.854784
2    b  one  0.306399 -0.154427
3    b  two  0.105416 -0.654746
4    a  one  0.373399  0.934213

In [2]:
#group by key1 
grouped = df.groupby(df['key1'])
grouped #this is now a more complex group object

In [3]:
#and generates a table per group
for name, group in grouped:
    print ("name:", name, "\n",group)

name: a 
   key1 key2     data1     data2
0    a  one  0.267529  0.828211
1    a  two  1.501438  1.854784
4    a  one  0.373399  0.934213
name: b 
   key1 key2     data1     data2
2    b  one  0.306399 -0.154427
3    b  two  0.105416 -0.654746


In [4]:
#access group table
grouped.get_group('b')

key1 key2     data1     data2
2    b  one  0.306399 -0.154427
3    b  two  0.105416 -0.654746

In [7]:
#get numper of entries (rows) per group
grouped.size()

key1
a    3
b    2
dtype: int64

In [8]:
#get number of group entries by columns
grouped.count()

key2  data1  data2
key1                    
a        3      3      3
b        2      2      2

#### Think of  grouped DataFrames as 3d objects:

In [9]:
#accessing the "3d" group tables
grouped['data2'].get_group('a')

0    0.828211
1    1.854784
4    0.934213
Name: data2, dtype: float64

In [10]:
grouped.get_group('a')['data2']

0    0.828211
1    1.854784
4    0.934213
Name: data2, dtype: float64

### Group by external keys

In [11]:
#define external key years as numpy array
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([years]).mean()


2005    0.624794
2006    0.339899
Name: data1, dtype: float64

### Group by functions

In [12]:
#sort by column and retun top n
def top(df, n=5, column='data1'):
    return df.sort_values(by=column)[-n:]

df.groupby(df['key1']).apply(top,n=5)

key1 key2     data1     data2
key1                                
a    0    a  one  0.267529  0.828211
     4    a  one  0.373399  0.934213
     1    a  two  1.501438  1.854784
b    3    b  two  0.105416 -0.654746
     2    b  one  0.306399 -0.154427

### Group-wise aggregation (apply)
<img SRC="IMG/groupby.jpg">

#### Typical build in aggregation functions:
* sum
* mean
* max / min
* quantile
* ...

In [13]:
#aggregate over the groups
grouped.count()

key2  data1  data2
key1                    
a        3      3      3
b        2      2      2

In [14]:
grouped.sum()

data1     data2
key1                    
a     2.142366  3.617208
b     0.411815 -0.809174

#### Custom Aggregation Functions

In [15]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

data1     data2
key1                    
a     1.233909  1.026573
b     0.200983  0.500319

#### Multiple aggregations

In [16]:
#just call a list of function
grouped.agg([peak_to_peak, 'mean', 'median'])

data1                            data2                    
     peak_to_peak      mean    median peak_to_peak      mean    median
key1                                                                  
a        1.233909  0.714122  0.373399     1.026573  1.205736  0.934213
b        0.200983  0.205908  0.205908     0.500319 -0.404587 -0.404587

#### Suppressing the Group Keys 

In [17]:
df.groupby(df['key1']).apply(top,n=2)

key1 key2     data1     data2
key1                                
a    4    a  one  0.373399  0.934213
     1    a  two  1.501438  1.854784
b    3    b  two  0.105416 -0.654746
     2    b  one  0.306399 -0.154427

In [18]:
df.groupby(df['key1'], group_keys=False).apply(top,n=2)

key1 key2     data1     data2
4    a  one  0.373399  0.934213
1    a  two  1.501438  1.854784
3    b  two  0.105416 -0.654746
2    b  one  0.306399 -0.154427

## More  Exercises in the Lab session... 